In [45]:
import sys
import os
import asyncio
from datetime import datetime
from typing import Optional, Dict, List, Any

# ========== 1. 1 设置项目路径 ==========
def setup_project_path():
    """设置项目路径"""
    print("=" * 70)
    print("🔧 Step 1.1: 设置项目路径")
    print("=" * 70)
    
    current_dir = os. getcwd()
    print(f"当前目录: {current_dir}")
    
    # 智能查找项目根目录
    if 'notebooks' in current_dir:
        project_root = os.path.dirname(current_dir)
    elif os.path.exists(os.path.join(current_dir, 'src')):
        project_root = current_dir
    else:
        project_root = current_dir
        for _ in range(3):
            if os.path.exists(os.path. join(project_root, 'src')):
                break
            project_root = os.path. dirname(project_root)
    
    # 添加到 Python 路径
    if project_root not in sys.path:
        sys.path. insert(0, project_root)
    
    # 切换工作目录
    os.chdir(project_root)
    
    print(f"项目根目录: {project_root}")
    print(f"工作目录: {os.getcwd()}")
    
    # 验证目录结构
    required_dirs = ['src', 'src/handlers', 'src/ai', 'src/services']
    for d in required_dirs: 
        path = os.path. join(project_root, d)
        status = "✅" if os.path. exists(path) else "❌"
        print(f"  {status} {d}")
    
    return project_root

PROJECT_ROOT = setup_project_path()

🔧 Step 1.1: 设置项目路径
当前目录: /Users/apapoo/Desktop/Github_Hub/SolumateBot
项目根目录: /Users/apapoo/Desktop/Github_Hub/SolumateBot
工作目录: /Users/apapoo/Desktop/Github_Hub/SolumateBot
  ✅ src
  ✅ src/handlers
  ✅ src/ai
  ✅ src/services


### ========== 1.2 加载环境变量 ==========

In [46]:
def load_environment():
    """加载环境变量"""
    print("\n" + "=" * 70)
    print("🔧 Step 1.2: 加载环境变量")
    print("=" * 70)
    
    env_file = os.path. join(PROJECT_ROOT, '.env')
    
    if not os.path.exists(env_file):
        print(f"❌ 未找到 .env 文件: {env_file}")
        return False
    
    from dotenv import load_dotenv
    load_dotenv(env_file, override=True)
    print(f"✅ 已加载:  {env_file}")
    
    # 显示关键配置（隐藏敏感信息）
    configs = {
        'DATABASE_URL': os. getenv('DATABASE_URL'),
        'TELEGRAM_BOT_TOKEN': os.getenv('TELEGRAM_BOT_TOKEN'),
        'OPENAI_API_KEY': os.getenv('OPENAI_API_KEY'),
        'ANTHROPIC_API_KEY': os.getenv('ANTHROPIC_API_KEY'),
        'VLLM_API_URL': os. getenv('VLLM_API_URL'),
    }
    
    print("\n📋 环境配置:")
    for key, value in configs.items():
        if value: 
            if 'KEY' in key or 'TOKEN' in key or 'URL' in key: 
                # 隐藏敏感信息
                if len(value) > 20:
                    safe_value = value[: 10] + "..." + value[-5:]
                else:
                    safe_value = value[: 5] + "..."
            else:
                safe_value = value[: 50] + "..." if len(value) > 50 else value
            print(f"  ✅ {key}: {safe_value}")
        else:
            print(f"  ⚠️ {key}: (未设置)")
    
    return True

ENV_OK = load_environment()


🔧 Step 1.2: 加载环境变量
✅ 已加载:  /Users/apapoo/Desktop/Github_Hub/SolumateBot/.env

📋 环境配置:
  ✅ DATABASE_URL: postgresql...umate
  ✅ TELEGRAM_BOT_TOKEN: 8203179267...R16c8
  ✅ OPENAI_API_KEY: your_opena..._here
  ✅ ANTHROPIC_API_KEY: your_anthr..._here
  ✅ VLLM_API_URL: https://ww...apis/


### ========== 1.3 验证依赖导入 ==========


In [47]:
def verify_imports():
    """验证所有依赖是否可以正常导入"""
    print("\n" + "=" * 70)
    print("🔧 Step 1.3: 验证依赖导入")
    print("=" * 70)
    
    imports_to_check = [
        ("telegram", "python-telegram-bot"),
        ("sqlalchemy", "SQLAlchemy"),
        ("openai", "OpenAI SDK"),
        ("anthropic", "Anthropic SDK"),
        ("aiohttp", "aiohttp"),
        ("loguru", "loguru"),
    ]
    
    all_ok = True
    for module_name, display_name in imports_to_check: 
        try:
            __import__(module_name)
            print(f"  ✅ {display_name}")
        except ImportError as e:
            print(f"  ❌ {display_name}: {e}")
            all_ok = False
    
    # 验证项目模块
    print("\n📦 项目模块:")
    project_modules = [
        "src.database",
        "src.models.database",
        "src.handlers.messages",
        "src.services.channel_manager",
        "src.services.message_router",
        "src.subscription.service",
        "src.ai.conversation",
        "config",
    ]
    
    for module in project_modules: 
        try: 
            __import__(module)
            print(f"  ✅ {module}")
        except Exception as e:
            print(f"  ❌ {module}: {e}")
            all_ok = False
    
    return all_ok

IMPORTS_OK = verify_imports()


🔧 Step 1.3: 验证依赖导入
  ✅ python-telegram-bot
  ✅ SQLAlchemy
  ✅ OpenAI SDK
  ✅ Anthropic SDK
  ✅ aiohttp
  ✅ loguru

📦 项目模块:
  ✅ src.database
  ✅ src.models.database
  ✅ src.handlers.messages
  ✅ src.services.channel_manager
  ✅ src.services.message_router
  ✅ src.subscription.service
  ✅ src.ai.conversation
  ✅ config


### 数据库连接与查看

In [48]:
print("\n" + "=" * 70)
print("🗄️ Step 2: 数据库连接与数据查看")
print("=" * 70)

DB_SESSION = None
DB_OK = False

def connect_database():
    """连接数据库并获取会话"""
    global DB_SESSION, DB_OK
    
    try:
        from src.database import get_db_session, engine
        from sqlalchemy import inspect, text
        
        # 测试连接
        with engine.connect() as conn:
            conn.execute(text("SELECT 1"))
        
        DB_SESSION = get_db_session()
        DB_OK = True
        
        # 显示表信息
        inspector = inspect(engine)
        tables = inspector.get_table_names()
        print(f"\n✅ 数据库连接成功!")
        print(f"📋 数据库表 ({len(tables)} 个): {', '.join(tables)}")
        
        return DB_SESSION
        
    except Exception as e:
        print(f"❌ 数据库连接失败:  {e}")
        import traceback
        traceback.print_exc()
        return None

def show_database_overview():
    """显示数据库概览"""
    if not DB_OK: 
        print("⚠️ 数据库未连接，跳过")
        return
    
    print("\n" + "-" * 50)
    print("📊 数据库概览")
    print("-" * 50)
    
    from src.models.database import User, Bot, Channel, ChannelBotMapping, Conversation
    
    stats = {
        "用户": db.query(User).count(),
        "机器人": db.query(Bot).count(),
        "频道": db. query(Channel).count(),
        "绑定关系": db.query(ChannelBotMapping).count(),
        "对话记录": db.query(Conversation).count(),
    }
    
    for name, count in stats. items():
        print(f"  {name}: {count}")

if DB_OK:
    show_database_overview()

    
    
db = connect_database()


🗄️ Step 2: 数据库连接与数据查看
2026-01-22 14:51:12,313 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-22 14:51:12,314 INFO sqlalchemy.engine.Engine SELECT 1
2026-01-22 14:51:12,314 INFO sqlalchemy.engine.Engine [cached since 1.08e+04s ago] {}
2026-01-22 14:51:12,316 INFO sqlalchemy.engine.Engine ROLLBACK
2026-01-22 14:51:12,318 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-22 14:51:12,319 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s]) AND pg_catalog.pg_class.relpersistence != %(relpersistence_1)s AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2026-01-22 14:51:12,319 INFO sqlalchemy.engine.Engine [cached since 1.08e+04s ago] {'param_1': 'r', 'param_2': 'p', 'relpersistence_1': 't', 'nspname_1': 'pg_cata

### 查看所有bot的配置

In [49]:
print("\n" + "=" * 70)
print("🤖 Step 3: Bot 配置详情")
print("=" * 70)

BOTS = []

def load_all_bots():
    """加载所有 Bot 配置"""
    global BOTS
    
    if not DB_OK: 
        print("⚠️ 数据库未连接，跳过")
        return []
    
    from src.models.database import Bot, User
    
    BOTS = db.query(Bot).all()
    
    if not BOTS: 
        print("⚠️ 暂无机器人配置")
        return []
    
    print(f"\n找到 {len(BOTS)} 个机器人:\n")
    
    for i, bot in enumerate(BOTS, 1):
        creator = db.query(User).filter(User.id == bot.created_by).first()
        creator_name = f"@{creator.username}" if creator else f"ID:{bot.created_by}"
        
        print(f"""
┌{'─' * 68}┐
│ Bot #{i}:  @{bot.bot_username: <56} │
├{'─' * 68}┤
│ 基本信息                                                              │
│   ID:           {bot.id:<53} │
│   UUID:         {bot. uuid:<53} │
│   名称:         {bot. bot_name:<53} │
│   状态:         {bot.status:<53} │
│   创建者:       {creator_name: <53} │
│   创建时间:     {str(bot.created_at):<53} │
├{'─' * 68}┤
│ AI 配置                                                               │
│   提供商:       {bot.ai_provider:<53} │
│   模型:          {bot.ai_model:<53} │
├{'─' * 68}┤
│ System Prompt (前200字符):                                            │
│   {(bot.system_prompt or '(未设置)')[:65]:<65} │
│   {(bot.system_prompt[65:130] if bot.system_prompt and len(bot.system_prompt) > 65 else ''):<65} │
│   {(bot.system_prompt[130:195] if bot.system_prompt and len(bot.system_prompt) > 130 else ''):<65} │
├{'─' * 68}┤
│ 其他设置:        {str(bot.settings or {})[:53]:<53} │
└{'─' * 68}┘""")
    
    return BOTS

BOTS = load_all_bots()


🤖 Step 3: Bot 配置详情
2026-01-22 14:51:15,037 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-22 14:51:15,039 INFO sqlalchemy.engine.Engine SELECT bots.id AS bots_id, bots.uuid AS bots_uuid, bots.bot_token AS bots_bot_token, bots.bot_name AS bots_bot_name, bots.bot_username AS bots_bot_username, bots.description AS bots_description, bots.personality AS bots_personality, bots.system_prompt AS bots_system_prompt, bots.ai_model AS bots_ai_model, bots.ai_provider AS bots_ai_provider, bots.settings AS bots_settings, bots.status AS bots_status, bots.is_public AS bots_is_public, bots.created_by AS bots_created_by, bots.version AS bots_version, bots.created_at AS bots_created_at, bots.updated_at AS bots_updated_at 
FROM bots
2026-01-22 14:51:15,039 INFO sqlalchemy.engine.Engine [cached since 1.075e+04s ago] {}

找到 1 个机器人:

2026-01-22 14:51:15,042 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.uuid AS users_uuid, users.telegram_id AS users_telegram_id, users.username AS u

### 查看频道与绑定关系

In [50]:
# =============================================================================
# 💬 Cell 4: 查看频道与绑定关系
# =============================================================================

print("\n" + "=" * 70)
print("💬 Step 4: 频道与绑定关系")
print("=" * 70)

CHANNELS = []
MAPPINGS = []

def load_channels_and_mappings():
    """加载频道和绑定关系"""
    global CHANNELS, MAPPINGS
    
    if not DB_OK: 
        print("⚠️ 数据库未连接，跳过")
        return [], []
    
    from src.models. database import Channel, ChannelBotMapping, Bot, User
    
    CHANNELS = db. query(Channel).all()
    MAPPINGS = db.query(ChannelBotMapping).all()
    
    if not CHANNELS: 
        print("⚠️ 暂无频道")
        return [], []
    
    print(f"\n找到 {len(CHANNELS)} 个频道, {len(MAPPINGS)} 个绑定关系:\n")
    
    for channel in CHANNELS: 
        owner = db.query(User).filter(User.id == channel. owner_id).first() if channel.owner_id else None
        
        print(f"""
┌{'─' * 68}┐
│ Channel ID: {channel.id:<56} │
├{'─' * 68}┤
│ Telegram Chat ID: {channel. telegram_chat_id:<49} │
│ 类型:            {channel.chat_type:<49} │
│ 标题:            {(channel.title or '(无)'):<49} │
│ 用户名:          {(channel.username or '(无)'):<49} │
│ 所有者:          {(f'@{owner. username}' if owner else '(无)'):<49} │
│ 状态:            {'✅ 激活' if channel. is_active else '❌ 未激活':<49} │
├{'─' * 68}┤
│ 绑定的机器人:                                                         │""")
        
        channel_mappings = [m for m in MAPPINGS if m.channel_id == channel.id]
        
        if channel_mappings:
            for m in channel_mappings:
                bot = db.query(Bot).filter(Bot.id == m. bot_id).first()
                bot_name = f"@{bot.bot_username}" if bot else f"Bot#{m.bot_id}"
                status = "✅" if m.is_active else "❌"
                print(f"│   {status} {bot_name: <20} | 模式: {m.routing_mode:<10} | 优先级: {m.priority:<5} │")
                if m.keywords:
                    print(f"│      关键词: {str(m.keywords)[:50]:<53} │")
        else:
            print(f"│   (无绑定)                                                          │")
        
        print(f"└{'─' * 68}┘")
    
    return CHANNELS, MAPPINGS

CHANNELS, MAPPINGS = load_channels_and_mappings()


💬 Step 4: 频道与绑定关系
2026-01-22 14:51:22,168 INFO sqlalchemy.engine.Engine SELECT channels.id AS channels_id, channels.telegram_chat_id AS channels_telegram_chat_id, channels.chat_type AS channels_chat_type, channels.title AS channels_title, channels.username AS channels_username, channels.owner_id AS channels_owner_id, channels.settings AS channels_settings, channels.subscription_tier AS channels_subscription_tier, channels.subscription_start_date AS channels_subscription_start_date, channels.subscription_end_date AS channels_subscription_end_date, channels.is_active AS channels_is_active, channels.version AS channels_version, channels.created_at AS channels_created_at, channels.updated_at AS channels_updated_at 
FROM channels
2026-01-22 14:51:22,169 INFO sqlalchemy.engine.Engine [cached since 1.072e+04s ago] {}


Exception during reset or similar
Traceback (most recent call last):
  File "/Users/apapoo/miniforge3/envs/solin/lib/python3.11/site-packages/sqlalchemy/engine/base.py", line 1969, in _exec_single_context
    self.dialect.do_execute(
  File "/Users/apapoo/miniforge3/envs/solin/lib/python3.11/site-packages/sqlalchemy/engine/default.py", line 922, in do_execute
    cursor.execute(statement, parameters)
psycopg2.OperationalError: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.


The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/apapoo/miniforge3/envs/solin/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/ls/fj0jr64131xc9pcgdjmpnl2w0000gp/T/ipykernel_39899/1106186787.py", line 64, in <module>
    CHANNELS, MAPPINGS = load_

/Users/apapoo/miniforge3/envs/solin/lib/python3.11/ast.py:50: RuntimeWarning: coroutine 'test_ai_conversation' was never awaited
  return compile(source, filename, mode, flags,
/Users/apapoo/miniforge3/envs/solin/lib/python3.11/ast.py:50: RuntimeWarning: coroutine 'simulate_full_message_flow' was never awaited
  return compile(source, filename, mode, flags,


OperationalError: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

[SQL: SELECT channels.id AS channels_id, channels.telegram_chat_id AS channels_telegram_chat_id, channels.chat_type AS channels_chat_type, channels.title AS channels_title, channels.username AS channels_username, channels.owner_id AS channels_owner_id, channels.settings AS channels_settings, channels.subscription_tier AS channels_subscription_tier, channels.subscription_start_date AS channels_subscription_start_date, channels.subscription_end_date AS channels_subscription_end_date, channels.is_active AS channels_is_active, channels.version AS channels_version, channels.created_at AS channels_created_at, channels.updated_at AS channels_updated_at 
FROM channels]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

### 消息路由服务调试

In [ ]:
print("\n" + "=" * 70)
print("🔀 Step 5: 消息路由服务调试")
print("=" * 70)

def test_message_router():
    """测试息路由服务"""
    if not DB_OK: 
        print("⚠️ 数据库未连接，跳过")
        return
    
    from src.services.message_router import MessageRouter
    from src.services.channel_manager import ChannelManagerService
    from src.models.database import Channel, ChannelBotMapping
    
    print("\n📌 5.1 测试 extract_mention (提取 @用户名)")
    print("-" * 50)
    
    test_messages = [
        "@SolinBot 你好啊",
        "你好 @SolinBot",
        "@Solin_AI_Bot 今天天气怎么样",
        "今天天气不错",
        "@test_bot, 请问.. .",
        "Hello @MyBot!",
        "@Bot1 @Bot2 多个机器人",
    ]
    
    for msg in test_messages:
        mention = MessageRouter.extract_mention(msg)
        print(f"  '{msg[: 40]}'")
        print(f"    -> 提取到: {mention or '(无)'}")
    
    print("\n📌 5.2 测试 should_respond_in_channel")
    print("-" * 50)
    
    # 获取一个有绑定的频道
    if CHANNELS and MAPPINGS:
        test_channel = CHANNELS[0]
        test_mappings = [m for m in MAPPINGS if m. channel_id == test_channel.id]
        
        chat_types = ['private', 'group', 'supergroup', 'channel']
        for chat_type in chat_types: 
            result = MessageRouter.should_respond_in_channel(chat_type, test_mappings)
            print(f"  chat_type='{chat_type}', mappings={len(test_mappings)} -> {result}")
        
        # 测试空 mappings
        result = MessageRouter.should_respond_in_channel('private', [])
        print(f"  chat_type='private', mappings=0 -> {result}")
    else:
        print("  ⚠️ 没有可用的频道和绑定进行测试")
    
    print("\n📌 5.3 测试 select_bot (选择响应的机器人)")
    print("-" * 50)
    
    if CHANNELS and MAPPINGS and BOTS:
        test_channel = CHANNELS[0]
        test_mappings = [m for m in MAPPINGS if m.channel_id == test_channel.id and m.is_active]
        
        # 加载 mapping 的 bot 对象
        for m in test_mappings:
            if not hasattr(m, 'bot') or m.bot is None:
                from src.models.database import Bot
                m.bot = db. query(Bot).filter(Bot.id == m.bot_id).first()
        
        if test_mappings: 
            bot_username = test_mappings[0].bot. bot_username if test_mappings[0].bot else "TestBot"
            
            test_cases = [
                (f"@{bot_username} 你好", bot_username, "有 @mention"),
                ("普通消息没有@", None, "无 @mention"),
                ("关键词测试", None, "关键词模式"),
            ]
            
            for msg_text, mention, desc in test_cases: 
                selected = MessageRouter.select_bot(
                    message_text=msg_text,
                    channel=test_channel,
                    mappings=test_mappings,
                    mentioned_username=mention
                )
                
                if selected: 
                    print(f"  [{desc}] '{msg_text[: 30]}'")
                    print(f"    -> 选中: @{selected.bot.bot_username} (mode={selected.routing_mode})")
                else:
                    print(f"  [{desc}] '{msg_text[:30]}'")
                    print(f"    -> 未选中任何 Bot")
    else:
        print("  ⚠️ 没有可用的数据进行测试")

if DB_OK:
    test_message_router()

### 订阅服务调试

In [ ]:
print("\n" + "=" * 70)
print("📊 Step 6: 订阅服务调试")
print("=" * 70)

def test_subscription_service():
    """测试订阅服务"""
    if not DB_OK:
        print("⚠️ 数据库未连接，跳过")
        return
    
    from src.subscription.service import SubscriptionService
    from src.models.database import User
    
    subscription_service = SubscriptionService(db)
    
    print("\n📌 6.1 获取/创建用户")
    print("-" * 50)
    
    # 使用实际的 Telegram ID 或测试 ID
    test_telegram_id = 7224312427  # 替换为你的 Telegram ID
    
    user = subscription_service. get_user_by_telegram_id(test_telegram_id)
    
    print(f"""
  用户信息:
    ID:               {user.id}
    UUID:            {user. uuid}
    Telegram ID:     {user.telegram_id}
    用户名:          @{user.username or '(无)'}
    名字:            {user. first_name or '(无)'}
    订阅等级:        {user.subscription_tier}
    订阅开始日期:    {user.subscription_start_date or '(无)'}
    订阅结束日期:    {user.subscription_end_date or '(无)'}
    是否激活:        {user.is_active}
""")
    
    print("\n📌 6.2 检查订阅状态")
    print("-" * 50)
    
    is_active = subscription_service.check_subscription_status(user)
    print(f"  订阅是否有效: {'✅ 是' if is_active else '❌ 否'}")
    
    print("\n📌 6.3 检查使用配额")
    print("-" * 50)
    
    can_send = subscription_service. check_usage_limit(user, action_type="message")
    print(f"  可以发送消息:  {'✅ 是' if can_send else '❌ 否 (配额已用完)'}")
    
    # 获取每日限额
    try:
        daily_limit = subscription_service.get_daily_limit(user. subscription_tier)
        print(f"  每日消息限额: {daily_limit}")
    except Exception as e:
        print(f"  每日消息限额: 获取失败 ({e})")
    
    print("\n📌 6.4 使用统计")
    print("-" * 50)
    
    try:
        stats = subscription_service. get_usage_stats(user)
        print(f"  今日消息使用: {stats. get('messages_used', 0)}/{stats.get('messages_limit', 0)}")
        print(f"  今日图片使用: {stats.get('images_used', 0)}")
    except Exception as e:
        print(f"  ⚠️ 获取统计失败:  {e}")
    
    return user

TEST_USER = None
if DB_OK: 
    TEST_USER = test_subscription_service()


### 对话服务调试

In [ ]:
print("\n" + "=" * 70)
print("🧠 Step 7: AI 对话服务调试")
print("=" * 70)

def test_ai_service():
    """测试 AI 对话服务"""
    
    print("\n📌 7.1 AI 配置信息")
    print("-" * 50)
    
    try:
        from config import settings
        
        print(f"""
  AI 提供商配置:
    vLLM API URL:       {settings.vllm_api_url or '(未配置)'}
    vLLM Model:        {getattr(settings, 'vllm_model', '(未配置)')}
    OpenAI API Key:     {'✅ 已配置' if settings.openai_api_key else '❌ 未配置'}
    OpenAI Model:      {getattr(settings, 'openai_model', 'gpt-4')}
    Anthropic API Key:  {'✅ 已配置' if settings.anthropic_api_key else '❌ 未配置'}
    Anthropic Model:    {getattr(settings, 'anthropic_model', 'claude-3-sonnet-20240229')}
""")
    except Exception as e: 
        print(f"  ❌ 获取配置失败:  {e}")
        return
    
    print("\n📌 7.2 初始化 AI 服务")
    print("-" * 50)
    
    try:
        from src.ai.conversation import conversation_service, ConversationService
        
        provider_name = type(conversation_service. provider).__name__
        print(f"  ✅ ConversationService 初始化成功")
        print(f"  当前使用的 Provider:  {provider_name}")
        
        # 显示 Provider 详情
        provider = conversation_service.provider
        if hasattr(provider, 'model'):
            print(f"  模型: {provider. model}")
        if hasattr(provider, 'api_url'):
            print(f"  API URL: {provider.api_url}")
            
    except Exception as e: 
        print(f"  ❌ 初始化失败: {e}")
        import traceback
        traceback.print_exc()
        return None
    
    return conversation_service

CONVERSATION_SERVICE = test_ai_service()

### 对话测试 (实际调用)

In [ ]:
print("\n" + "=" * 70)
print("🎯 Step 8: AI 对话测试 (实际调用 LLM)")
print("=" * 70)

async def test_ai_conversation():
    """实际测试 AI 对话"""
    if CONVERSATION_SERVICE is None:
        print("⚠️ AI 服务未初始化，跳过")
        return
    
    print("\n📌 8.1 基本对话测试")
    print("-" * 50)
    
    # 测试消息
    test_message = "你好，请介绍一下你自己"
    
    print(f"  📨 用户消息: {test_message}")
    print(f"  ⏳ 正在调用 AI...")
    
    try:
        import time
        start_time = time. time()
        
        response = await CONVERSATION_SERVICE.get_response(
            user_message=test_message,
            conversation_history=[]
        )
        
        elapsed_time = time.time() - start_time
        
        print(f"  ✅ 响应时间: {elapsed_time:.2f} 秒")
        print(f"\n  🤖 AI 回复:")
        print("  " + "-" * 60)
        # 格式化输出
        for line in response.split('\n'):
            print(f"  {line}")
        print("  " + "-" * 60)
        
    except Exception as e: 
        print(f"  ❌ 调用失败: {e}")
        import traceback
        traceback.print_exc()
    
    print("\n📌 8.2 带历史对话测试")
    print("-" * 50)
    
    # 带历史的对话
    history = [
        {"role": "user", "content": "我叫小明"},
        {"role":  "assistant", "content": "你好小明！很高兴认识你！"},
    ]
    
    test_message_2 = "你还记得我叫什么名字吗？"
    
    print(f"  📜 对话历史:")
    for msg in history: 
        role = "👤" if msg["role"] == "user" else "🤖"
        print(f"    {role}:  {msg['content']}")
    
    print(f"\n  📨 用户消息: {test_message_2}")
    print(f"  ⏳ 正在调用 AI...")
    
    try: 
        import time
        start_time = time.time()
        
        response = await CONVERSATION_SERVICE.get_response(
            user_message=test_message_2,
            conversation_history=history
        )
        
        elapsed_time = time.time() - start_time
        
        print(f"  ✅ 响应时间: {elapsed_time:.2f} 秒")
        print(f"\n  🤖 AI 回复:")
        print("  " + "-" * 60)
        for line in response.split('\n'):
            print(f"  {line}")
        print("  " + "-" * 60)
        
    except Exception as e: 
        print(f"  ❌ 调用失败: {e}")

# 运行异步测试
if CONVERSATION_SERVICE: 
    print("\n⏳ 开始 AI 对话测试...")
#     asyncio.run(test_ai_conversation())
    await test_ai_conversation()

else:
    print("⚠️ AI 服务未初始化，跳过对话测试")


In [ ]:
print("\n📌 8.3 带 System Prompt 测试")
print("-" * 50)

# 使用第一个 Bot 的 system prompt
if BOTS:
    test_bot = BOTS[0]
    system_prompt = test_bot.system_prompt

    if system_prompt: 
        print(f"  📝 使用 @{test_bot. bot_username} 的 System Prompt")
        print(f"     (前100字符): {system_prompt[: 100]}...")

        history_with_system = [
            {"role": "system", "content": system_prompt}
        ]

        test_message_3 = "今天心情有点低落"

        print(f"\n  📨 用户消息: {test_message_3}")
        print(f"  ⏳ 正在调用 AI...")
        if 1:
#         try:
            import time
            start_time = time.time()

            response = await CONVERSATION_SERVICE.get_response(
                user_message=test_message_3,
                conversation_history=history_with_system
            )

            elapsed_time = time. time() - start_time

            print(f"  ✅ 响应时间:  {elapsed_time:.2f} 秒")
            print(f"\n  🤖 AI 回复 (使用人设):")
            print("  " + "-" * 60)
            for line in response.split('\n'):
                print(f"  {line}")
            print("  " + "-" * 60)
#         except Exception as e: 
#             print(f"  ❌ 调用失败: {e}")

    else:
        print(f"  ⚠️ @{test_bot. bot_username} 没有设置 System Prompt")
else:
    print("  ⚠️ 没有可用的 Bot")


### 完整消息处理流程模拟

In [ ]:
print("\n" + "=" * 70)
print("🔄 Step 9: 完整消息处理流程模拟")
print("=" * 70)

async def simulate_full_message_flow(
    telegram_user_id: int,
    chat_id: int,
    chat_type: str,
    message_text: str,
    mentioned_bot:  str = None
):
    """
    模拟完整的消息处理流程
    
    Args:
        telegram_user_id: Telegram 用户 ID
        chat_id:  Telegram 聊天 ID
        chat_type: 聊天类型 (private/group/channel)
        message_text: 用户消息内容
        mentioned_bot: @的机器人用户名 (可选)
    """
    print(f"""
╔══════════════════════════════════════════════════════════════════════════════╗
║                           完整消息处理流程模拟                                 ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  输入参数:                                                                    ║
║    Telegram User ID:  {telegram_user_id: <57} ║
║    Chat ID:           {chat_id: <57} ║
║    Chat Type:        {chat_type: <57} ║
║    Message:           {message_text[: 50]:<57} ║
║    Mentioned Bot:     {(mentioned_bot or '(无)'):<57} ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")
    
    if not DB_OK: 
        print("❌ 数据库未连接，无法继续")
        return
    
    from src.database import get_db_session
    from src.services.channel_manager import ChannelManagerService
    from src.services. message_router import MessageRouter
    from src.subscription.service import SubscriptionService
    from src.models.database import Conversation, Bot
    
    db = get_db_session()
    results = {}
    
    try:
        # ========== Phase 1: 频道处理 ==========
        print("┌" + "─" * 68 + "┐")
        print("│ Phase 1: 频道管理                                                    │")
        print("└" + "─" * 68 + "┘")
        
        channel_service = ChannelManagerService(db)
        
        print(f"  📍 1.1 查找/创建 Channel...")
        channel = channel_service.get_or_create_channel(
            telegram_chat_id=chat_id,
            chat_type=chat_type,
            owner_id=telegram_user_id
        )
        print(f"      ✅ Channel ID: {channel.id}")
        print(f"      Chat Type: {channel. chat_type}")
        results['channel'] = channel
        
        print(f"\n  📍 1.2 获取频道绑定的 Bot...")
        mappings = channel_service.get_channel_bots(channel.id, active_only=True)
        print(f"      ✅ 找到 {len(mappings)} 个活跃绑定")
        
        for m in mappings: 
            bot = db. query(Bot).filter(Bot.id == m.bot_id).first()
            m.bot = bot  # 确保 bot 对象已加载
            print(f"         @{bot.bot_username}:  mode={m.routing_mode}, priority={m.priority}")
        
        results['mappings'] = mappings
        
        # ========== Phase 2: 消息路由 ==========
        print("\n┌" + "─" * 68 + "┐")
        print("│ Phase 2: 消息路由                                                    │")
        print("└" + "─" * 68 + "┘")
        
        print(f"  📍 2.1 检查是否应该响应...")
        should_respond = MessageRouter.should_respond_in_channel(chat_type, mappings)
        print(f"      结果: {'✅ 应该响应' if should_respond else '❌ 不响应'}")
        
        if not should_respond: 
            print("\n  ⚠️ 不需要响应，流程结束")
            return results
        
        print(f"\n  📍 2.2 提取 @mention...")
        mentioned_username = MessageRouter.extract_mention(message_text)
        print(f"      提取到: {mentioned_username or '(无)'}")
        
        print(f"\n  📍 2.3 选择响应的 Bot...")
        selected_mapping = MessageRouter.select_bot(
            message_text=message_text,
            channel=channel,
            mappings=mappings,
            mentioned_username=mentioned_username
        )
        
        if not selected_mapping: 
            print("      ❌ 没有选中任何 Bot")
            print("\n  ⚠️ 没有 Bot 响应，流程结束")
            return results
        
        selected_bot = selected_mapping.bot
        print(f"      ✅ 选中: @{selected_bot.bot_username}")
        print(f"         路由模式: {selected_mapping.routing_mode}")
        print(f"         优先级: {selected_mapping.priority}")
        results['selected_bot'] = selected_bot
        results['selected_mapping'] = selected_mapping
        
        # ========== Phase 3: 用户验证 ==========
        print("\n┌" + "─" * 68 + "┐")
        print("│ Phase 3: 用户验证与配额检查                                          │")
        print("└" + "─" * 68 + "┘")
        
        subscription_service = SubscriptionService(db)
        
        print(f"  📍 3.1 获取/创建用户...")
        db_user = subscription_service.get_user_by_telegram_id(telegram_user_id)
        print(f"      ✅ 用户 ID: {db_user.id}")
        print(f"         用户名: @{db_user. username or '(无)'}")
        print(f"         订阅等级: {db_user.subscription_tier}")
        results['user'] = db_user
        
        print(f"\n  📍 3.2 检查订阅状态...")
        is_subscription_active = subscription_service.check_subscription_status(db_user)
        print(f"      结果: {'✅ 订阅有效' if is_subscription_active else '❌ 订阅已过期'}")
        
        if not is_subscription_active: 
            print("\n  ⚠️ 订阅已过期，流程结束")
            return results
        
        print(f"\n  📍 3.3 检查使用配额...")
        can_send = subscription_service.check_usage_limit(db_user, action_type="message")
        print(f"      结果: {'✅ 配额充足' if can_send else '❌ 配额已用完'}")
        
        if not can_send: 
            print("\n  ⚠️ 配额已用完，流程结束")
            return results
        
        # 获取使用统计
        try:
            stats = subscription_service.get_usage_stats(db_user)
            print(f"      今日已用: {stats.get('messages_used', 0)}/{stats.get('messages_limit', 0)}")
        except:
            pass
        
        # ========== Phase 4: 获取对话历史 ==========
        print("\n┌" + "─" * 68 + "┐")
        print("│ Phase 4: 获取对话历史                                                │")
        print("└" + "─" * 68 + "┘")
        
        print(f"  📍 4.1 查询最近对话记录...")
        recent_conversations = db. query(Conversation).filter(
            Conversation.user_id == db_user.id
        ).order_by(Conversation.timestamp.desc()).limit(10).all()
        
        print(f"      ✅ 找到 {len(recent_conversations)} 条历史记录")
        
        print(f"\n  📍 4.2 构建对话历史...")
        history = []
        for conv in reversed(recent_conversations):
            if conv.is_user_message:
                history. append({"role": "user", "content": conv. message})
            else: 
                history.append({"role": "assistant", "content": conv.response})
        
        # 添加 System Prompt
        if selected_bot.system_prompt:
            history.insert(0, {"role": "system", "content": selected_bot.system_prompt})
            print(f"      ✅ 添加 System Prompt ({len(selected_bot.system_prompt)} 字符)")
        
        print(f"      ✅ 对话历史:  {len(history)} 条消息")
        
        # 显示历史
        if history:
            print(f"\n      📜 对话历史预览:")
            for i, msg in enumerate(history[-5:]):  # 只显示最后5条
                role = msg["role"]
                content = msg["content"][: 50] + "..." if len(msg["content"]) > 50 else msg["content"]
                role_icon = {"system": "⚙️", "user":  "👤", "assistant":  "🤖"}.get(role, "❓")
                print(f"         {role_icon} [{role}]: {content}")
        
        results['history'] = history
        
        # ========== Phase 5: AI 生成回复 ==========
        print("\n┌" + "─" * 68 + "┐")
        print("│ Phase 5: AI 生成回复                                                 │")
        print("└" + "─" * 68 + "┘")
        
        if CONVERSATION_SERVICE is None:
            print("  ⚠️ AI 服务未初始化，使用模拟回复")
            ai_response = f"[模拟回复] 你说的是:  {message_text}"
        else: 
            print(f"  📍 5.1 调用 AI 服务...")
            print(f"      Provider: {type(CONVERSATION_SERVICE. provider).__name__}")
            print(f"      用户消息: {message_text}")
            print(f"      ⏳ 正在生成回复...")
            
            import time
            start_time = time.time()
            
            try:
                ai_response = await CONVERSATION_SERVICE.get_response(
                    user_message=message_text,
                    conversation_history=history
                )
                
                elapsed_time = time. time() - start_time
                print(f"      ✅ 生成成功!  耗时:  {elapsed_time:. 2f} 秒")
                print(f"      回复长度: {len(ai_response)} 字符")
                
            except Exception as e: 
                print(f"      ❌ AI 调用失败: {e}")
                ai_response = f"[错误] AI 服务调用失败: {str(e)}"
        
        print(f"\n  📍 5.2 AI 回复内容:")
        print("  " + "─" * 60)
        for line in ai_response.split('\n')[:10]:  # 最多显示10行
            print(f"  {line}")
        if ai_response.count('\n') > 10:
            print(f"  ... (共 {ai_response.count(chr(10)) + 1} 行)")
        print("  " + "─" * 60)
        
        results['ai_response'] = ai_response
        
        # ========== Phase 6: 保存记录 (模拟) ==========
        print("\n┌" + "─" * 68 + "┐")
        print("│ Phase 6: 保存记录 (模拟，不实际写入)                                  │")
        print("└" + "─" * 68 + "┘")
        
        print(f"  📍 6.1 [模拟] 保存用户消息到 conversations 表...")
        print(f"      user_id: {db_user.id}")
        print(f"      message:  {message_text[: 50]}...")
        print(f"      is_user_message:  True")
        
        print(f"\n  📍 6.2 [模拟] 保存 Bot 回复到 conversations 表...")
        print(f"      user_id: {db_user.id}")
        print(f"      response: {ai_response[:50]}...")
        print(f"      is_user_message: False")
        
        print(f"\n  📍 6.3 [模拟] 记录使用量到 usage_records 表...")
        print(f"      user_id: {db_user.id}")
        print(f"      action_type: message")
        print(f"      count: 1")
        
        print(f"\n  📍 6.4 [模拟] 提交事务...")
        print(f"      ⏭️ 跳过 (调试模式)")
        
        # ========== 完成 ==========
        print("\n" + "═" * 70)
        print("✅ 消息处理流程模拟完成!")
        print("═" * 70)
        
        print(f"""
📋 流程总结:
   ├── Channel:  ID={channel.id} ({channel.chat_type})
   ├── Bot: @{selected_bot.bot_username} (mode={selected_mapping.routing_mode})
   ├── User: ID={db_user. id} (@{db_user. username or '无'})
   ├── 对话历史: {len(history)} 条
   ├── AI 回复: {len(ai_response)} 字符
   └── 状态: ✅ 成功
""")
        
        return results
        
    except Exception as e:
        print(f"\n❌ 流程出错: {e}")
        import traceback
        traceback.print_exc()
        return results
    
    finally: 
        db.close()


# 运行完整流程模拟
print("\n⏳ 开始完整流程模拟...")

# 配置测试参数
TEST_PARAMS = {
    "telegram_user_id": 7224312427,      # 替换为你的 Telegram ID
    "chat_id": 7224312427,               # 私聊时等于 user_id
    "chat_type": "private",
    "message_text": "今天天气怎么样？我想出去走走",
}

# 现在可以正常运行了
if DB_OK:
    import nest_asyncio
    nest_asyncio.apply()
    flow_results = asyncio.run(simulate_full_message_flow(**TEST_PARAMS))
else:
    print("⚠️ 数据库未连接，跳过流程模拟")

### 多场景测试

In [ ]:
print("\n" + "=" * 70)
print("🧪 Step 10: 多场景测试")
print("=" * 70)

async def run_multiple_scenarios():
    """运行多个测试场景"""
    if not DB_OK or CONVERSATION_SERVICE is None:
        print("⚠️ 环境未就绪，跳过多场景测试")
        return
    
    scenarios = [
        {
            "name":  "场景1: 简单问候",
            "message": "你好",
            "expected":  "应该返回友好的问候"
        },
        {
            "name": "场景2: 情感倾诉",
            "message": "我今天心情有点低落，工作压力很大",
            "expected": "应该返回安慰和支持的回复"
        },
        {
            "name": "场景3: 问题咨询",
            "message": "如何提高工作效率？",
            "expected": "应该返回有帮助的建议"
        },
        {
            "name": "场景4: 日常闲聊",
            "message": "你喜欢什么颜色？",
            "expected": "应该返回个性化的回答"
        },
    ]
    
    print(f"\n将测试 {len(scenarios)} 个场景:\n")
    
    for i, scenario in enumerate(scenarios, 1):
        print(f"┌{'─' * 68}┐")
        print(f"│ {scenario['name']: <65} │")
        print(f"├{'─' * 68}┤")
        print(f"│ 输入: {scenario['message'][: 60]: <61} │")
        print(f"│ 期望:  {scenario['expected'][:60]:<61} │")
        print(f"└{'─' * 68}┘")
        
        try:
            import time
            start_time = time.time()
            
            # 使用第一个 Bot 的 system prompt
            history = []
            if BOTS and BOTS[0].system_prompt:
                history.append({"role": "system", "content":  BOTS[0].system_prompt})
            
            response = await CONVERSATION_SERVICE.get_response(
                user_message=scenario['message'],
                conversation_history=history
            )
            
            elapsed_time = time.time() - start_time
            
            print(f"  ⏱️ 耗时: {elapsed_time:.2f}s | 回复长度: {len(response)} 字符")
            print(f"  🤖 回复: {response[: 100]}...")
            print()
            
        except Exception as e: 
            print(f"  ❌ 错误: {e}")
            print()
        
        # 避免 API 限流
        await asyncio.sleep(1)
    
    print("✅ 多场景测试完成!")

if DB_OK and CONVERSATION_SERVICE: 
    print("\n⏳ 开始多场景测试...")
    asyncio. run(run_multiple_scenarios())

### 性能分析

In [ ]:
print("\n" + "=" * 70)
print("📈 Step 11: 性能分析")
print("=" * 70)

async def performance_analysis():
    """性能分析"""
    if CONVERSATION_SERVICE is None:
        print("⚠️ AI 服务未初始化，跳过性能分析")
        return
    
    import time
    
    print("\n📍 11.1 AI 响应时间测试 (5次)")
    print("-" * 50)
    
    test_message = "你好"
    times = []
    
    for i in range(5):
        start = time.time()
        try:
            await CONVERSATION_SERVICE. get_response(
                user_message=test_message,
                conversation_history=[]
            )
            elapsed = time. time() - start
            times.append(elapsed)
            print(f"  第 {i+1} 次: {elapsed:.2f} 秒")
        except Exception as e:
            print(f"  第 {i+1} 次: 失败 ({e})")
        
        await asyncio.sleep(0.5)
    
    if times:
        print(f"\n  📊 统计:")
        print(f"     平均响应时间: {sum(times)/len(times):.2f} 秒")
        print(f"     最快:  {min(times):.2f} 秒")
        print(f"     最慢: {max(times):.2f} 秒")
    
    if DB_OK:
        print("\n📍 11.2 数据库查询性能")
        print("-" * 50)
        
        from src.database import get_db_session
        from src.models.database import User, Conversation
        
        db = get_db_session()
        
        # 测试用户查询
        start = time.time()
        for _ in range(10):
            db. query(User).filter(User.telegram_id == 7224312427).first()
        user_query_time = (time.time() - start) / 10
        print(f"  用户查询 (平均): {user_query_time*1000:.2f} ms")
        
        # 测试对话历史查询
        start = time.time()
        for _ in range(10):
            db.query(Conversation).filter(
                Conversation.user_id == 1
            ).order_by(Conversation.timestamp.desc()).limit(10).all()
        conv_query_time = (time. time() - start) / 10
        print(f"  对话历史查询 (平均): {conv_query_time*1000:.2f} ms")
        
        db.close()

if CONVERSATION_SERVICE: 
    print("\n⏳ 开始性能分析...")
    asyncio.run(performance_analysis())



### 自定义测试

In [22]:
print("\n" + "=" * 70)
print("🔧 Step 12: 自定义测试函数")
print("=" * 70)

async def custom_chat(message:  str, system_prompt: str = None, history: list = None):
    """
    自定义对话测试
    
    Args: 
        message: 用户消息
        system_prompt:  自定义 system prompt (可选)
        history: 对话历史 (可选)
    
    Example:
        await custom_chat("你好", system_prompt="你是一个幽默的助手")
    """
    if CONVERSATION_SERVICE is None:
        print("❌ AI 服务未初始化")
        return
    
    print(f"\n📨 用户消息: {message}")
    
    conversation_history = history or []
    
    if system_prompt: 
        conversation_history. insert(0, {"role": "system", "content": system_prompt})
        print(f"📝 System Prompt: {system_prompt[: 50]}...")
    
    print("⏳ 正在生成回复...")
    
    try: 
        import time
        start = time.time()
        
        response = await CONVERSATION_SERVICE.get_response(
            user_message=message,
            conversation_history=conversation_history
        )
        
        elapsed = time.time() - start
        print(f"⏱️ 耗时: {elapsed:.2f} 秒")
        print(f"\n🤖 AI 回复:")
        print("-" * 50)
        print(response)
        print("-" * 50)
        
        return response
        
    except Exception as e: 
        print(f"❌ 错误: {e}")
        return None


def quick_db_query(table_name: str, limit: int = 10):
    """
    快速数据库查询
    
    Args: 
        table_name:  表名 (users/bots/channels/conversations/...)
        limit: 返回记录数
    
    Example:
        quick_db_query("users")
        quick_db_query("conversations", limit=5)
    """
    if not DB_OK: 
        print("❌ 数据库未连接")
        return
    
    from src.database import get_db_session
    from src.models.database import User, Bot, Channel, Conversation, ChannelBotMapping
    
    table_map = {
        "users": User,
        "bots":  Bot,
        "channels": Channel,
        "conversations":  Conversation,
        "mappings": ChannelBotMapping,
        "channel_bot_mappings": ChannelBotMapping,
    }
    
    if table_name. lower() not in table_map: 
        print(f"❌ 未知的表: {table_name}")
        print(f"   可用的表: {list(table_map.keys())}")
        return
    
    model = table_map[table_name.lower()]
    db = get_db_session()
    
    try:
        records = db.query(model).limit(limit).all()
        print(f"\n📋 {table_name} 表 (前 {limit} 条):")
        print("-" * 50)
        
        for record in records:
            print(f"  {record}")
        
        total = db.query(model).count()
        print(f"\n总记录数: {total}")
        
    finally:
        db.close()


print("""
📌 可用的自定义测试函数: 

1. custom_chat(message, system_prompt=None, history=None)
   - 自定义对话测试
   - 示例:  await custom_chat("你好", system_prompt="你是一个幽默的助手")

2. quick_db_query(table_name, limit=10)
   - 快速数据库查询
   - 示例: quick_db_query("users")
   - 示例:  quick_db_query("conversations", limit=5)

在 Jupyter 中使用:
   >>> await custom_chat("介绍一下自己")
   >>> quick_db_query("bots")
""")



🔧 Step 12: 自定义测试函数

📌 可用的自定义测试函数: 

1. custom_chat(message, system_prompt=None, history=None)
   - 自定义对话测试
   - 示例:  await custom_chat("你好", system_prompt="你是一个幽默的助手")

2. quick_db_query(table_name, limit=10)
   - 快速数据库查询
   - 示例: quick_db_query("users")
   - 示例:  quick_db_query("conversations", limit=5)

在 Jupyter 中使用:
   >>> await custom_chat("介绍一下自己")
   >>> quick_db_query("bots")



### 测试总结

In [27]:
print("\n" + "=" * 70)
print("📋 Step 13: 测试总结")
print("=" * 70)

print(f"""
╔══════════════════════════════════════════════════════════════════════════════╗
║                              测试结果总结                                      ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  环境配置                                                                     ║
║    ├── 项目路径:         {'✅ 正确' if PROJECT_ROOT else '❌ 失败': <53} ║
║    ├── 环境变量:        {'✅ 已加载' if ENV_OK else '❌ 失败': <53} ║
║    ├── 依赖导入:        {'✅ 正常' if IMPORTS_OK else '❌ 失败':<53} ║
║    └── 数据库连接:       {'✅ 成功' if DB_OK else '❌ 失败':<53} ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  数据状态                                                                     ║
║    ├── 机器人数量:      {len(BOTS) if BOTS else 0:<53} ║
║    ├── 频道数量:        {len(CHANNELS) if CHANNELS else 0:<53} ║
║    └── 绑定关系数量:    {len(MAPPINGS) if MAPPINGS else 0:<53} ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  AI 服务                                                                      ║
║    └── 状态:        {'✅ 已初始化' if CONVERSATION_SERVICE else '❌ 未初始化': <53} ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")

if all([ENV_OK, IMPORTS_OK, DB_OK, CONVERSATION_SERVICE]):
    print("🎉 所有组件正常，Bot 可以正常工作!")
else:
    print("⚠️ 部分组件异常，请检查上述失败项")


📋 Step 13: 测试总结

╔══════════════════════════════════════════════════════════════════════════════╗
║                              测试结果总结                                      ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  环境配置                                                                     ║
║    ├── 项目路径:         ✅ 正确                                                  ║
║    ├── 环境变量:        ✅ 已加载                                                 ║
║    ├── 依赖导入:        ❌ 失败                                                  ║
║    └── 数据库连接:       ✅ 成功                                                  ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  数据状态                                                                     ║
║    ├── 机器人数量:      1                                                     ║
║    ├── 频道数量:        2                                                     ║
║    └── 绑定关系数量:    2                 